In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab


## TWEETSENT

In [11]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [12]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [13]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [14]:
text_Tweetsent = df_tweet['text']
polarity = np.asarray(df_tweet['sentiment'])

In [15]:
X_text = count_vect.transform(text_Tweetsent)

X_text_transform = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_text_transform.shape

(11533, 25852)

In [16]:
X_train1, X_test_TWEET, Y_train1, Y_test_TWEET = train_test_split(X_text_transform, polarity, test_size=0.3)

In [17]:
X_train12, X_test_TWEET1, Y_train12, Y_test_TWEET1 = train_test_split(df_tweet['text'], df_tweet['sentiment'] , test_size=0.3)

## TASH

In [18]:
df = pd.read_csv("tash-pt.csv")
df = df.dropna()
df = df.drop(columns=['id_twitter'])


In [19]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [20]:
text_TASH = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

In [21]:
X_train2, X_test_TASH, Y_train2, Y_test_TASH = train_test_split(X_text_TASH_, sentiment, test_size=0.3)

In [22]:
X_train21, X_test_TASH21, Y_train21, Y_test_TASH21 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## KANSOAN

In [23]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [24]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [25]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

In [26]:
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [27]:
X_train31, X_test_KANSOAN1, Y_train31, Y_test_KANSOAN1 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## TEST

In [3]:
df = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [4]:
df_all = df_all.dropna()

In [5]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [6]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [7]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [8]:
count_vect = CountVectorizer()

Tweet = count_vect.fit_transform(Tweet)

tfidf_transformer = TfidfTransformer()
X_train_transform = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_transform.shape

(12626, 25852)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)



In [10]:
X_train_text, X_test_text, Y_train_text, Y_test_text = train_test_split(df_all['Tweet'] , df_all['Polaridade'], test_size=0.3)


## NB - TRAIN -> KANSOAN - TEST -> TWEETSENT

In [28]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1015
           0       0.26      0.63      0.37       869
           1       0.60      0.53      0.56      1576

    accuracy                           0.40      3460
   macro avg       0.29      0.39      0.31      3460
weighted avg       0.34      0.40      0.35      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> TASH

In [29]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       262
           0       0.37      0.71      0.49       305
           1       0.44      0.41      0.42       270

    accuracy                           0.39       837
   macro avg       0.27      0.38      0.30       837
weighted avg       0.28      0.39      0.32       837



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> UNILEX

In [30]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

#sentiment = df['sentiment'].apply(func)
#X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [31]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1279
         0.0       0.32      0.48      0.38      1398
         1.0       0.34      0.52      0.41      1111

    accuracy                           0.33      3788
   macro avg       0.55      0.33      0.26      3788
weighted avg       0.55      0.33      0.26      3788



## SVM - TRAIN->UNILEX - TEST->TWEETSENT

In [32]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1015
           0       0.26      0.51      0.34       869
           1       0.54      0.61      0.57      1576

    accuracy                           0.40      3460
   macro avg       0.27      0.37      0.31      3460
weighted avg       0.31      0.40      0.35      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM T U T TASH

In [33]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       262
           0       0.39      0.63      0.48       305
           1       0.39      0.49      0.44       270

    accuracy                           0.39       837
   macro avg       0.26      0.37      0.30       837
weighted avg       0.27      0.39      0.32       837



## SVM T KANSOAN T unilex


In [34]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1279
         0.0       0.29      0.30      0.29      1398
         1.0       0.30      0.63      0.41      1111

    accuracy                           0.30      3788
   macro avg       0.53      0.31      0.23      3788
weighted avg       0.53      0.30      0.23      3788



## RL T U T TWEETSENT

In [35]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1015
           0       0.26      0.46      0.33       869
           1       0.54      0.65      0.59      1576

    accuracy                           0.41      3460
   macro avg       0.26      0.37      0.30      3460
weighted avg       0.31      0.41      0.35      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T U T TASH

In [36]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       262
           0       0.38      0.59      0.46       305
           1       0.38      0.51      0.44       270

    accuracy                           0.38       837
   macro avg       0.25      0.37      0.30       837
weighted avg       0.26      0.38      0.31       837



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T U T KANSOAN

In [37]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1279
         0.0       0.26      0.20      0.22      1398
         1.0       0.30      0.73      0.42      1111

    accuracy                           0.29      3788
   macro avg       0.52      0.31      0.22      3788
weighted avg       0.52      0.29      0.21      3788



## TOTALMENT.C T U T TWEETSENT

In [64]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=['accuracy']
)

one = OneHotEncoder(sparse=False)

y_one = np.asarray(Y_train3)
y_one_test = Y_test_TWEET

fit = model.fit(X_train3.todense(), y_one, epochs=10, validation_data=(X_test_TWEET.todense(), y_one_test))

Epoch 1/10
202/202 [==============================] - 2s 12ms/step - loss: 0.3129 - accuracy: 0.9185 - val_loss: 0.5183 - val_accuracy: 0.4110
Epoch 2/10
202/202 [==============================] - 2s 10ms/step - loss: 0.0597 - accuracy: 0.9816 - val_loss: 0.4626 - val_accuracy: 0.4020
Epoch 3/10
202/202 [==============================] - 2s 9ms/step - loss: 0.0289 - accuracy: 0.9918 - val_loss: 0.5545 - val_accuracy: 0.4035
Epoch 4/10
202/202 [==============================] - 2s 10ms/step - loss: 0.0157 - accuracy: 0.9963 - val_loss: 0.4747 - val_accuracy: 0.3855
Epoch 5/10
202/202 [==============================] - 2s 8ms/step - loss: 0.0119 - accuracy: 0.9966 - val_loss: 0.7924 - val_accuracy: 0.4130
Epoch 6/10
202/202 [==============================] - 2s 8ms/step - loss: 0.0098 - accuracy: 0.9964 - val_loss: 0.5879 - val_accuracy: 0.3890
Epoch 7/10
202/202 [==============================] - 2s 8ms/step - loss: 0.0085 - accuracy: 0.9964 - val_loss: 0.9572 - val_accuracy: 0.4145
Epo

In [65]:
predicted = model.predict(X_test_TWEET.todense())
for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1
        
print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.40      0.67      0.50      1026
           0       0.00      0.00      0.00       859
           1       0.57      0.62      0.59      1575

    accuracy                           0.48      3460
   macro avg       0.32      0.43      0.36      3460
weighted avg       0.38      0.48      0.42      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LSTM T U T TWEETSENT

In [38]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df['text'])))

vocab = set_array(df['text'])

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [67]:
text_KANSOAN

X_train3_, X_test_KANSOAN_, Y_train3_, Y_test_KANSOAN_ = train_test_split(text_KANSOAN, df['sentiment'], test_size=0.3)

In [68]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = np.asarray(Y_train3_)
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 6s 122ms/step - loss: 0.6481 - accuracy: 0.6248 - val_loss: 0.7494 - val_accuracy: 0.3986
Epoch 2/10
51/51 [==============================] - 5s 95ms/step - loss: 0.4691 - accuracy: 0.8017 - val_loss: 0.7795 - val_accuracy: 0.3829
Epoch 3/10
51/51 [==============================] - 4s 87ms/step - loss: 0.3337 - accuracy: 0.8700 - val_loss: 0.9439 - val_accuracy: 0.4092
Epoch 4/10
51/51 [==============================] - 4s 88ms/step - loss: 0.2560 - accuracy: 0.9006 - val_loss: 0.8315 - val_accuracy: 0.3986
Epoch 5/10
51/51 [==============================] - 4s 87ms/step - loss: 0.2206 - accuracy: 0.9133 - val_loss: 1.0586 - val_accuracy: 0.4147
Epoch 6/10
51/51 [==============================] - 4s 87ms/step - loss: 0.1997 - accuracy: 0.9221 - val_loss: 0.7731 - val_accuracy: 0.3829
Epoch 7/10
51/51 [==============================] - 4s 87ms/step - loss: 0.1847 - accuracy: 0.9274 - val_loss: 1.1426 - val_accuracy: 0.4078
Epoch 8/10
5

In [79]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.36      0.62      0.46       990
           0       0.00      0.00      0.00       898
           1       0.52      0.58      0.55      1572

    accuracy                           0.44      3460
   macro avg       0.29      0.40      0.34      3460
weighted avg       0.34      0.44      0.38      3460



## UNILEX

In [86]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.36      0.63      0.46      1269
         0.0       0.00      0.00      0.00      1416
         1.0       0.34      0.48      0.40      1103

    accuracy                           0.35      3788
   macro avg       0.23      0.37      0.28      3788
weighted avg       0.22      0.35      0.27      3788



## TASH

In [88]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.38      0.61      0.47       272
           0       0.00      0.00      0.00       305
           1       0.35      0.54      0.42       260

    accuracy                           0.37       837
   macro avg       0.24      0.38      0.30       837
weighted avg       0.23      0.37      0.28       837



## CONV1D T U T TWEET

In [89]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = Y_train3_
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 1s 25ms/step - loss: 0.6273 - accuracy: 0.6529 - val_loss: 0.8887 - val_accuracy: 0.4165
Epoch 2/10
51/51 [==============================] - 1s 20ms/step - loss: 0.3837 - accuracy: 0.8251 - val_loss: 0.7291 - val_accuracy: 0.4168
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.2657 - accuracy: 0.8851 - val_loss: 0.7881 - val_accuracy: 0.4306
Epoch 4/10
51/51 [==============================] - 1s 21ms/step - loss: 0.2156 - accuracy: 0.9081 - val_loss: 0.9029 - val_accuracy: 0.4283
Epoch 5/10
51/51 [==============================] - 1s 22ms/step - loss: 0.1838 - accuracy: 0.9257 - val_loss: 0.9450 - val_accuracy: 0.4240
Epoch 6/10
51/51 [==============================] - 1s 21ms/step - loss: 0.1663 - accuracy: 0.9311 - val_loss: 0.7601 - val_accuracy: 0.4043
Epoch 7/10
51/51 [==============================] - 1s 19ms/step - loss: 0.1525 - accuracy: 0.9354 - val_loss: 0.7565 - val_accuracy: 0.3968
Epoch 8/10
51

In [90]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.38      0.55      0.45       990
           0       0.00      0.00      0.00       898
           1       0.52      0.68      0.59      1572

    accuracy                           0.46      3460
   macro avg       0.30      0.41      0.35      3460
weighted avg       0.35      0.46      0.40      3460



## UNILEX

In [91]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.38      0.63      0.48      1269
         0.0       0.00      0.00      0.00      1416
         1.0       0.34      0.52      0.41      1103

    accuracy                           0.37      3788
   macro avg       0.24      0.39      0.30      3788
weighted avg       0.23      0.37      0.28      3788



## TASH

In [92]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.38      0.53      0.44       272
           0       0.00      0.00      0.00       305
           1       0.34      0.59      0.43       260

    accuracy                           0.35       837
   macro avg       0.24      0.37      0.29       837
weighted avg       0.23      0.35      0.28       837



## BDR T U T TWEET

In [93]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = Y_train3_
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 3s 53ms/step - loss: 0.6928 - accuracy: 0.5170 - val_loss: 0.7002 - val_accuracy: 0.4543
Epoch 2/10
51/51 [==============================] - 1s 18ms/step - loss: 0.6913 - accuracy: 0.5147 - val_loss: 0.7146 - val_accuracy: 0.4543
Epoch 3/10
51/51 [==============================] - 1s 20ms/step - loss: 0.6869 - accuracy: 0.5147 - val_loss: 0.7266 - val_accuracy: 0.4543
Epoch 4/10
51/51 [==============================] - 1s 19ms/step - loss: 0.6774 - accuracy: 0.5536 - val_loss: 0.7249 - val_accuracy: 0.4549
Epoch 5/10
51/51 [==============================] - 1s 19ms/step - loss: 0.6568 - accuracy: 0.7261 - val_loss: 0.7224 - val_accuracy: 0.4283
Epoch 6/10
51/51 [==============================] - 1s 19ms/step - loss: 0.6259 - accuracy: 0.8065 - val_loss: 0.7273 - val_accuracy: 0.4165
Epoch 7/10
51/51 [==============================] - 1s 18ms/step - loss: 0.5888 - accuracy: 0.8315 - val_loss: 0.7368 - val_accuracy: 0.4142
Epoch 8/10
51

In [94]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.36      0.55      0.44       990
           0       0.00      0.00      0.00       898
           1       0.52      0.64      0.57      1572

    accuracy                           0.45      3460
   macro avg       0.29      0.40      0.34      3460
weighted avg       0.34      0.45      0.38      3460



## UNILEX

In [95]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.34      0.78      0.48      1269
         0.0       0.00      0.00      0.00      1416
         1.0       0.33      0.27      0.30      1103

    accuracy                           0.34      3788
   macro avg       0.22      0.35      0.26      3788
weighted avg       0.21      0.34      0.25      3788



## TASH

In [96]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.39      0.58      0.46       272
           0       0.00      0.00      0.00       305
           1       0.35      0.58      0.44       260

    accuracy                           0.37       837
   macro avg       0.25      0.39      0.30       837
weighted avg       0.23      0.37      0.29       837

